In [278]:
# Setting
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.actions.action_builder import ActionBuilder
from selenium.webdriver.common.actions.mouse_button import MouseButton
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

url = "https://pcmap.place.naver.com/restaurant/13166754/review/visitor"

In [279]:
# 크롬창 열기
options = Options()
driver = webdriver.Chrome(options = options)

# 창모드 전체화면으로 크기 늘리기
driver.maximize_window()

# 로딩시간: 페이지 로딩
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_all_elements_located((By.TAG_NAME, 'body')))

# 네이버 리뷰 주소 접속하기
driver.get(url)

# 로딩시간: 페이지 로딩
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_all_elements_located((By.TAG_NAME, 'body')))

[<selenium.webdriver.remote.webelement.WebElement (session="362868224bb6e8afe8615e4595a8b229", element="f.AEC597B2DB553F81BDA4C3C14856A5AB.d.F6BC0B1666F57E013B2BBD233FCAEB7E.e.31")>]

In [283]:
rows = driver.find_elements(By.CLASS_NAME, 'owAeM')
# # 아이디
# user_id = rows[0].find_element(By.CLASS_NAME, 'qgLL3')

# # 리뷰 텍스트
# review_text = rows[0].find_element(By.CLASS_NAME, 'zPfVt')
# print(review_text.text)
# # 방문 횟수
# number_visits = rows[5].find_elements(By.CLASS_NAME, 'CKUdu')[1]
# # 방문 날짜
# visit_date = rows[10].find_elements(By.CLASS_NAME, 'CKUdu')[0].find_elements(By.CLASS_NAME, 'place_blind')[1]

- 리뷰내용 더보기 버튼을 누르는 것에 문제가 생김

In [103]:
# 해결
# 리뷰 내용 더보기 버튼 클릭
# review_text_extend = rows[4].find_element(By.CLASS_NAME, 'Ky28p').click()
review_extend = driver.find_element(By.CLASS_NAME, 'fvwqf').click()

In [192]:
# 데이터를 받을 빈 딕셔너리
# data = {"UserID":[],"Review_Text":[], "NumberOfVisit":[]}

# 더보기 한번당 리뷰 10개씩
# 몇개의 리뷰를 확인해야하나
# 200개의 리뷰 확인
for temp in range(19):
    driver.find_element(By.CLASS_NAME, 'fvwqf').click()
    time.sleep(1)


In [176]:
temp_rows = driver.find_elements(By.TAG_NAME, 'li')

In [194]:
len(rows)

210

In [195]:
temp_list = []
for i in range(200):
    user_id = rows[i].find_element(By.CLASS_NAME, 'qgLL3')
    temp_list.append(user_id.text)

In [196]:
len(temp_list)

200

- 200개 이상이 나오게 한 뒤 200개까지만 추출하는 방법
- 200개가 안나오는 것에 대한 오류 확인
- 200개 잘나오는 거 확인
    - 안나왔던 이유
    - rows를 for문을 돌린뒤 한번 더 실행시켜 확인을 시켜줘야 했다.

In [280]:
# 데이터 받을 빈 딕셔너리
review_data = {"UserID":[],"Review_Text":[],"Date":[], "NumberOfVisit":[]}

# 리뷰 200개까지 나오게 미리 더보기 클릭하기
# for문 19번 돌리는 이유는 처음에 10개 있는 것을 빼고 세야함
print("총 리뷰 개수 구하기")
for num_reviews in range(19):
    driver.find_element(By.CLASS_NAME, 'fvwqf').click()
    time.sleep(1)
rows = driver.find_elements(By.CLASS_NAME, 'owAeM')

# rows의 개수를 세어 for문을 돌린다.
cnt = len(rows)
print("총 리뷰 개수: ", cnt)

# 리뷰 추출하기
log_count = 0
print("START", end = " >>> ")
for i in range(cnt):
    # 리뷰 내용 더보기 버튼이 없는 리뷰들 예외구문
    try:
        review_text_extend = rows[i].find_element(By.CLASS_NAME, 'Ky28p').click()
    except:
        pass
    user_id = rows[i].find_element(By.CLASS_NAME, 'qgLL3')
    review_text = rows[i].find_element(By.CLASS_NAME, 'zPfVt')
    number_visits = rows[i].find_elements(By.CLASS_NAME, 'CKUdu')[1]
    visit_date = rows[i].find_elements(By.CLASS_NAME, 'CKUdu')[0].find_elements(By.CLASS_NAME, 'place_blind')[1]
    
    review_data['UserID'].append(user_id.text)
    review_data['Review_Text'].append(review_text.text)
    review_data['Date'].append(visit_date.text)
    review_data['NumberOfVisit'].append(number_visits.text)
    
    # 로그 만들기
    log_count += 1
    if log_count > 0 and log_count % 20 == 0:
        print(log_count, end= " >>> ")
print("End")

print(review_data)

리뷰 추출하기 시작
총 리뷰 개수:  200
START >>> 20 >>> 40 >>> 60 >>> 80 >>> 100 >>> 120 >>> 140 >>> 160 >>> 180 >>> 200 >>> End


- 리뷰 갯수가 for문 range를 19로 설정하면 원래라면 200이 되어야 하지만 처음 더보기 버튼 누르기 전에 리뷰 10개가 있으므로 총 210개가 된다.
- 하지만 지금 range 설정이 19로 되어있으면 190개가 되거나 혹은 200개가 되어있다. range 설정을 20으로 하면 200개가 되거나 210개가 된다.
- 그리고 날짜도 표에 넣어야할거 같다.

In [271]:
title = driver.find_element(By.CLASS_NAME, 'GHAhO').text
print(title)

서울신라호텔 더 파크뷰


In [282]:
# 데이터프레임 만들기
df_reviews = pd.DataFrame(review_data)

# .csv파일로 저장
df_reviews.to_csv("./shilla_hotel_buffet_review_original.csv")

---

- 함수로 만들어 정리해준다.

In [352]:
# Setting
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.actions.action_builder import ActionBuilder
from selenium.webdriver.common.actions.mouse_button import MouseButton
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd


In [358]:
# Functions

def open_url(url):
    """크롬 창을 열고 url에 접속하는 함수"""
    # 크롬창 열기
    options = Options()
    driver = webdriver.Chrome(options = options)

    # 창모드 전체화면으로 크기 늘리기
    driver.maximize_window()

    # 로딩시간: 페이지 로딩
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_all_elements_located((By.TAG_NAME, 'body')))

    # 네이버 리뷰 주소 접속하기
    driver.get(url)

    # 로딩시간: 페이지 로딩
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_all_elements_located((By.TAG_NAME, 'body')))
    
    return driver

def count_reviews(driver):
    """더보기 버튼을 통해 구하고자 하는 리뷰 갯수의 총 갯수를 구하는 함수"""
    for num_reviews in range(19):
        driver.find_element(By.CLASS_NAME, 'fvwqf').click()
        time.sleep(1.5)
    rows = driver.find_elements(By.CLASS_NAME, 'owAeM')

    # rows의 개수를 세어 for문을 돌린다.
    cnt = len(rows)
    print("총 리뷰 개수:", cnt)
    
    return rows, cnt

In [359]:
def crawling_reviews(rows, cnt):
    """리뷰 데이터를 추출하는 함수"""
    # 데이터 받을 빈 딕셔너리
    review_data = {"UserID":[],"Review_Text":[],"Date":[], "NumberOfVisit":[]}
    
    # 리뷰 추출하기
    log_count = 0
    print("START", end = " >>> ")
    for i in range(cnt):
        # 리뷰 내용 더보기 버튼이 없는 리뷰들 예외구문
        try:
            rows[i].find_element(By.CLASS_NAME, 'Ky28p').click()
        except:
            pass
        user_id = rows[i].find_element(By.CLASS_NAME, 'qgLL3')
        review_text = rows[i].find_element(By.CLASS_NAME, 'zPfVt')
        number_visits = rows[i].find_elements(By.CLASS_NAME, 'CKUdu')[1]
        visit_date = rows[i].find_elements(By.CLASS_NAME, 'CKUdu')[0].find_elements(By.CLASS_NAME, 'place_blind')[1]
        
        review_data['UserID'].append(user_id.text)
        review_data['Review_Text'].append(review_text.text)
        review_data['Date'].append(visit_date.text)
        review_data['NumberOfVisit'].append(number_visits.text)
        
        # 로그 만들기
        log_count += 1
        if log_count > 0 and log_count % 20 == 0:
            print(log_count, end= " >>> ")
    print("End")
    
    return review_data

In [360]:
# Setting
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.actions.action_builder import ActionBuilder
from selenium.webdriver.common.actions.mouse_button import MouseButton
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

def get_review(url):
    """메인 함수"""
    # 크롬 창 열기
    print(f"크롬 창을 열고 있습니다.", end=" ")
    driver = open_url(url)
    print()
    
    title = driver.find_element(By.CLASS_NAME, 'GHAhO').text

    # 총 리뷰 개수 구하기
    print("'{0}'의 리뷰를 가지고 오는 중입니다.".format(title))
    rows, cnt = count_reviews(driver)
    
    # 딕셔너리 만들기
    review_data = crawling_reviews(rows,cnt)
    
    # 데이터프레임 만들기
    print("데이터프레임으로 변환 중입니다.",end=" ")
    df_reviews = pd.DataFrame(review_data)
    print("... 완료!")
    
    # .csv파일로 저장
    df_reviews.to_csv("./shilla_hotel_buffet_review_original.csv")
    
    driver.quit()
    
    return df_reviews

In [361]:
url = "https://pcmap.place.naver.com/restaurant/13166754/review/visitor"

shilla_hotel_buffet_review = get_review(url)

크롬 창을 열고 있습니다. 
'서울신라호텔 더 파크뷰'의 리뷰를 가지고 오는 중입니다.
총 리뷰 개수: 200
START >>> 20 >>> 40 >>> 60 >>> 80 >>> 100 >>> 120 >>> 140 >>> 160 >>> 180 >>> 200 >>> End
데이터프레임으로 변환 중입니다. ... 완료!
